<a href="https://colab.research.google.com/github/Saba-Gul/Chat-With-PDF-Retrieval-Augmented-Generation-RAG-Implementation/blob/main/RAG_ChatwithPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chat With PDF:**
This code loads a PDF document, processes it into smaller chunks, and stores the embeddings in a vector store. It then performs a similarity search to retrieve relevant document chunks based on a query, formats the context and question into a prompt, and generates a response using OpenAI's language model. The process is structured to handle large documents efficiently and provide accurate answers based on the content of the PDF.

![image.png](https://i0.wp.com/www.phdata.io/wp-content/uploads/2023/11/image1-3.png)

credit:https://www.phdata.io/blog/what-is-retrieval-augmented-generation-rag/

In [1]:
!pip install openai langchain chromadb tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0

In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.5 MB/s eta 0:00:00


In [13]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 13.5 MB/s eta 0:00:00


In [3]:
import langchain
import openai
import chromadb # vector store for storing dense embeddings
import tiktoken # tokenization for text encoding

In [6]:
from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')

# **Load and ingest data into pipline**

In [14]:
from langchain.document_loaders import PyPDFLoader
loader= PyPDFLoader('/content/attentionisallyouneed.pdf')
pages = loader.load()

In [15]:
pages[1].page_content

'1 Introduction\nRecurrent neural networks, long short-term memory [ 13] and gated recurrent [ 7] neural networks\nin particular, have been firmly established as state of the art approaches in sequence modeling and\ntransduction problems such as language modeling and machine translation [ 35,2,5]. Numerous\nefforts have since continued to push the boundaries of recurrent language models and encoder-decoder\narchitectures [38, 24, 15].\nRecurrent models typically factor computation along the symbol positions of the input and output\nsequences. Aligning the positions to steps in computation time, they generate a sequence of hidden\nstates ht, as a function of the previous hidden state ht−1and the input for position t. This inherently\nsequential nature precludes parallelization within training examples, which becomes critical at longer\nsequence lengths, as memory constraints limit batching across examples. Recent work has achieved\nsignificant improvements in computational efficiency th

In [16]:
len(pages)

15

# **Preprocessing**

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(pages)

In [18]:
len(docs)

49

Tiktoken is a swift BPE (Byte Pair Encoding) tokenizer designed specifically for OpenAI's models

In [19]:
import tiktoken
encoding = tiktoken.encoding_for_model("text-embedding-3-small")

doc_tokens = [len(encoding.encode(page.page_content)) for page in docs]

In [20]:
doc_tokens

[254,
 206,
 167,
 64,
 184,
 181,
 191,
 180,
 156,
 211,
 202,
 215,
 209,
 167,
 254,
 200,
 213,
 116,
 230,
 227,
 194,
 175,
 198,
 207,
 208,
 189,
 380,
 245,
 217,
 96,
 439,
 218,
 209,
 60,
 341,
 205,
 229,
 119,
 313,
 276,
 279,
 190,
 278,
 296,
 305,
 184,
 245,
 295,
 282]

In [21]:
total_tokens = sum(doc_tokens)
print(f"Total tokens: {total_tokens}")

Total tokens: 10699


In [22]:
cost = total_tokens * 0.0004
print(f"Cost: ${cost}")

Cost: $4.2796


# **Vector Embeddings**

In [30]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings


In [31]:
embedding_function=OpenAIEmbeddings(model="text-embedding-3-small",api_key=openai.api_key)

Initializes a Chroma vector store by creating embeddings for a set of documents using a specified embedding function, and it persists these embeddings in a specified directory for future use.

In [32]:
db=Chroma.from_documents(documents=docs,embedding=embedding_function, persist_directory='my-embeddings')

# **Similarity Search**

In [33]:
results=db.similarity_search_with_relevance_scores('What are self attention??',k=5)

for (doc,score) in results:
  print('score',score)
  print(doc)
  print('-------------------')

score 0.33666051995451773
page_content='described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehension, abstractive summarization,
textual entailment and learning task-independent sentence representations [4, 27, 28, 22].
End-to-end memory networks are based on a recurrent attention mechanism instead of sequence-
aligned recurrence and have been shown to perform well on simple-language question answering and
language modeling tasks [34].
To the best of our knowledge, however, the Transformer is the first transduction model relying
entirely on self-attention to compute representations of its input and output without using sequence-
aligned RNNs or convolution. In the following sections, we will describe the Transformer, motivate' metadata={'page': 1, 's

# **RAG Enriched Prompt**

In [34]:
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI

In [35]:
question='Please give me an introduction to transformer architecture'

context_docs=db.similarity_search(question,k=5)

#Create a prompt with two variables 'context ' and 'question'
prompt=PromptTemplate(
    template=""""Use the following pieces of context to answer the questions at the end.If
    you don't know the answer, just say don't know. do not try to make up the answer.

    <context>
    {context}
    </context>

    Question: {question}
    Helpful Answer""",
    input_variables=["context","question"]
)

llm=ChatOpenAI(model='gpt-4o-mini',temperature=0.9,api_key=openai.api_key)
qa_chain=LLMChain(llm=llm,prompt=prompt)

result=qa_chain(
    {'question':question,
     'context':"/n".join([doc.page_content for doc in context_docs])}
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
